In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from gamma_surfaces import get_gamma_surface, show_master_gamma_surface, show_gamma_surface_fit
from utilities import make_structure

## Generate s7-tlB-gb DFT expansion series

In [4]:
import copy

from atomistic.bicrystal import GammaSurface
from castep_parse import write_input_files as write_castep_input_files, read_output_files as read_castep_output
import numpy as np
from plotly import graph_objects

from utilities import get_castep_parameters

#### Find the minimum shift from the Lammps gamma surface

In [4]:
structure_code = 's7-tlB-gb'
lattice_parameters = 'eam'

gs = get_gamma_surface(structure_code, lattice_parameters)
show_master_gamma_surface(gs)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

In [5]:
min_coords = gs.get_minimum_fitted_shift('energy')
min_coords

[GammaSurfaceCoordinate(shift=array([0. , 0.4]), expansion=-1.0, energy=-541.1941043757),
 GammaSurfaceCoordinate(shift=array([0. , 0.4]), expansion=0.0, energy=-548.1553787659),
 GammaSurfaceCoordinate(shift=array([0. , 0.4]), expansion=1.0, energy=-545.545174989),
 GammaSurfaceCoordinate(shift=array([0. , 0.4]), expansion=2.0, energy=-537.4554113305)]

In [6]:
show_gamma_surface_fit(gs, min_coords[0].shift)

FigureWidget({
    'data': [{'name': 'Fit',
              'type': 'scatter',
              'uid': '9aee3681-0d…

#### Generate a structure using the DFT lattice parameters

In [10]:
bicrystal = make_structure('s7-tlB-gb', 'sized', 'dft')
bicrystal.show(include={'points': ['atoms']})

layout_args: None
visual_args: {'include': {'points': ['atoms']}}


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'supercell',
              'type': 'scatte…

#### Generate a gamma surface (just expansions at the Lammps-predicted minimum shift)

In [12]:
dft_exp = GammaSurface(
    bicrystal,
    shifts=[min_coords[0].shift for _ in range(3)],
#     expansions=[-0.1, 0.0, 0.1, 0.2,] # first set
    expansions=[0.3, 0.4, 0.5], # second set
)

In [13]:
dft_exp.shifts

array([[0. , 0.4],
       [0. , 0.4],
       [0. , 0.4]])

In [14]:
dft_exp.expansions

array([0.3, 0.4, 0.5])

#### Generate CASTEP input files

In [17]:
base_path = Path(r'C:\code_local\scratch\_castep_sims\expansion\s7-tlB-gb\1')
pot_base_dir = Path('.').resolve()
input_paths = []

common_castep_params = get_castep_parameters(bicrystal.num_atoms)

In [18]:
for i in dft_exp.all_coordinates():
    
    # Make a directory for this sim:
    sim_path = base_path.joinpath(i.coordinate_fmt)
    sim_path.mkdir()
    
    # Write simulation inputs:
    castep_params = {
        'supercell': i.structure.supercell,
        'atom_sites': i.structure.atoms.coords,
        'species': i.structure.atoms.labels['species'].unique_values,
        'species_idx': i.structure.atoms.labels['species'].values_idx,
        'dir_path': sim_path,
        **copy.deepcopy(common_castep_params),
    }
    input_paths.append(write_castep_input_files(**castep_params))    

#### Collate simulation outputs

In [13]:
base_dir = Path(r'C:\Users\adamj\Dropbox (The University of Manchester)\sims_db\castep\s7-tlB-gb\expansions_series\0\sims')

simulated_exp_params = {
    'shifts': [],
    'expansions': [],
    'data': {
        'energy': []
    },
}
for i in base_dir.glob('*'):
        
    print(i.name)
    
    shift_str, exp_str = i.name.split('__')
    shift = []
    for j in shift_str.split('_'):
        num, denom = j.split('.')
        shift.append(int(num) / int(denom))
        
    simulated_exp_params['shifts'].append(shift)
    simulated_exp_params['expansions'].append(float(exp_str))
    
    cst_out = read_castep_output(dir_path=i)
    simulated_exp_params['data']['energy'].append(cst_out['final_energy'][-1])

0.1_2.5__+0.000
0.1_2.5__+0.100
0.1_2.5__+0.200
0.1_2.5__-0.100


In [14]:
simulated_exp = GammaSurface(bicrystal, **simulated_exp_params)

In [17]:
simulated_exp.add_fit('energy', fit_size=3)

In [21]:
fit_plot_dat = simulated_exp.get_fit_plot_data('energy', [0, 0.4])
fig = graph_objects.FigureWidget(
    data=[
        {
            **fit_plot_dat['fitted_data'],
            'name': 'Fit',
        },
        {
            **fit_plot_dat['data'],
            'name': 'energy',
        },
        {
            **fit_plot_dat['minimum'],
            'name': 'Fit min.',
        },
    ],
    layout={
        'xaxis': {
            'title': 'Expansion',
        },
        'yaxis': {
            'title': 'energy',
        },
        'width': 400,
        'height': 400,
    }
)
fig

FigureWidget({
    'data': [{'name': 'Fit',
              'type': 'scatter',
              'uid': 'f3f9374e-c3…

# Visualise gamma surface data

The code below uses JSON data files that are stored within this repository to visualise the fitted gamma surfaces.

In [26]:
structure_code = 's7-tlB-gb'
lattice_parameters = 'eam'

gs = get_gamma_surface(structure_code, lattice_parameters)
show_master_gamma_surface(gs)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

In [27]:
# Show quadratic fitting at a given shift:
show_gamma_surface_fit(gs, shift=[0, 0.4])

FigureWidget({
    'data': [{'name': 'Fit',
              'type': 'scatter',
              'uid': 'f25ae581-6c…

# Method of gamma surface generation and analysis

This section breaks down the steps used to:

- generate gamma surface input files (for LAMMPS)
- run the simulations
- collate the results
- perform quadratic fitting
- visualise the master gamma surface

> **Note**: you cannot run simulations on Binder!

In [6]:
from subprocess import run, PIPE
import numpy as np

from atomistic.bicrystal import GammaSurface
from lammps_parse import write_lammps_inputs, read_lammps_output
from plotly import graph_objects

from utilities import get_lammps_parameters

# Change this variable to the directory where LAMMPS gamma surface simulation input files should be generated:
LAMMPS_SIMS_DIR = Path('C:\code_local\scratch\lammps_sims')

# Change this to the LAMMPS executable name (note, you cannot run simulations on Binder!):
LAMMPS_EXECUTABLE = 'lmp_serial'

**Generate a bicrystal whose gamma surface is to be explored**

In [9]:
bicrystal = make_structure('s7-tlA-gb', lattice_parameters='eam', configuration='sized')
bicrystal.show(
    include={'points': ['atoms']},
    layout_args={'height': 800},
)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'supercell',
              'type': 'scatte…

**Generate a gamma surface from a grid**

In [10]:
gamma_surface = GammaSurface.from_grid(bicrystal, grid=[3, 9], expansions=[0, 1, 2])

**Iterate over all coordinates in the gamma surface to generate simulation input files**

In [13]:
pot_base_dir = Path('.').resolve()
input_paths = []

common_lammps_params = get_lammps_parameters(base_path=pot_base_dir)

for i in gamma_surface.all_coordinates():
    
    # Make a directory for this sim:
    sim_path = LAMMPS_SIMS_DIR.joinpath(i.coordinate_fmt)
    sim_path.mkdir()
    
    # Write simulation inputs:
    lammps_params = {
        'supercell': i.structure.supercell,
        'atom_sites': i.structure.atoms.coords,
        'species': i.structure.atoms.labels['species'].unique_values,
        'species_idx': i.structure.atoms.labels['species'].values_idx,
        'dir_path': sim_path,
        **common_lammps_params,
    }
    input_paths.append(write_lammps_inputs(**lammps_params))
    

**Run simulations**

In [14]:
for i in input_paths:
    cmd = '{} < {}'.format(LAMMPS_EXECUTABLE, i.name)
    proc = run(cmd, shell=True, cwd=i.parent , stdout=PIPE, stderr=PIPE)    

**Collate simulation outputs**

In [15]:
simulated_gamma_surface_params = {
    'shifts': [],
    'expansions': [],
    'data': {
        'energy': []
    },
}
for i in input_paths:
        
    shift_str, exp_str = i.parent.name.split('__')    
    shift = []
    for j in shift_str.split('_'):
        num, denom = j.split('.')
        shift.append(int(num) / int(denom))
        
    simulated_gamma_surface_params['shifts'].append(shift)
    simulated_gamma_surface_params['expansions'].append(float(exp_str))
    
    lammps_out = read_lammps_output(dir_path=i.parent)
    simulated_gamma_surface_params['data']['energy'].append(
        lammps_out['final_energy'][-1])

**Generate a new GammaSurface to represent the simulated data**

In [16]:
simulated_gamma_surface = GammaSurface(bicrystal, **simulated_gamma_surface_params)

**Plot a slice of the gamma surface**

In [17]:
plot_dat = simulated_gamma_surface.get_surface_plot_data('energy', expansion=0, xy_as_grid=False)
grid_dat = simulated_gamma_surface.get_xy_plot_data()
graph_objects.FigureWidget(
    data=[
        {
            'type': 'contour',
            'colorscale': 'viridis',
            **plot_dat,            
        },
        {
            **grid_dat,
            'mode': 'markers',
            'marker': {
                'size': 2,
            },
        },
    ],
    layout={
        'xaxis': {
            'scaleanchor': 'y',          
        },        
    }
)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

**Fit at each shift to find the master gamma surface, and plot**

In [18]:
simulated_gamma_surface.add_fit('energy', 3)

In [19]:
master_plot_data = simulated_gamma_surface.get_fitted_surface_plot_data('energy', xy_as_grid=False)
graph_objects.FigureWidget(
    data=[
        {
            'type': 'contour',
            'colorscale': 'viridis',
            **master_plot_data,            
        },
        {
            **grid_dat,
            'mode': 'markers',
            'marker': {
                'size': 2,
            },
        },        
    ],
    layout={
        'xaxis': {
            'scaleanchor': 'y',
        }
    }
)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

**Plot the fit at a given shift**

In [20]:
fit_plot_dat = simulated_gamma_surface.get_fit_plot_data('energy', [0, 0])
fig = graph_objects.FigureWidget(
    data=[
        {
            **fit_plot_dat['fitted_data'],
            'name': 'Fit',
        },
        {
            **fit_plot_dat['data'],
            'name': 'energy',
        },
        {
            **fit_plot_dat['minimum'],
            'name': 'Fit min.',
        },
    ],
    layout={
        'xaxis': {
            'title': 'Expansion',
        },
        'yaxis': {
            'title': 'energy',
        },
        'width': 400,
        'height': 400,
    }
)
fig

FigureWidget({
    'data': [{'name': 'Fit',
              'type': 'scatter',
              'uid': '88702aac-80…

# Using wrapper functions in `gamma_surfaces.py`

The above workflow is wrapped up into a few wrapper functions in the python file `gamma_surfaces.py`.

In [21]:
from gamma_surfaces import compute_master_gamma_surface

In [23]:
structure_code = 's7-tlA-gb'
sims_dir = LAMMPS_SIMS_DIR.joinpath(structure_code)
sims_dir.mkdir()
gamma_surface = compute_master_gamma_surface(structure_code, sims_dir) # 5 mins run time

In [24]:
show_master_gamma_surface(gamma_surface)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …